##### Cleaning dataset for training

In [10]:
import pandas as pd
import os
import numpy as np

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/woba modeling/data/')
df = pd.read_csv('pitch_23_25_w.csv')

##### Spray Angle

In [11]:
spray_radian = np.arctan((df['hc_x'] - 125.42) / (198.27 - df['hc_y'])) # calcuate spray angle by setting plate as origin calcaluating angle from opp over adj
df['spray_angle'] = spray_radian * 180 / np.pi * .75

##### only select relevant features based on project aims

In [12]:
df['year'] = pd.to_datetime(df['game_date']).dt.year
df = df[['year', 'description', 'batter', 'woba_value', 'zone', 'b_stands', 'launch_speed', 'launch_angle', 'swing_path_tilt', 'attack_angle', 'attack_direction', 'spray_angle']]

##### drop any na's in 2023 bc bat tracking

In [13]:
df = df[~((df['year'] == 2023) & (df.isna().any(axis=1)))]

##### map for catagorical feature

In [14]:
df = df[df['zone'].notna()]
df['zone'] = df['zone'].astype(int)

swing path tilt

In [15]:
df.loc[df['b_stands'] == 'R', 'attack_direction'] *= -1
df.loc[df['b_stands'] == 'R', 'spray_angle'] *= -1

##### export

In [16]:
df.to_csv('pitch_cleaned.csv')